# Tutorial notebook

This notebook describes how to train Neural Network Regressors and Extreme Gradient Boosted Regression Trees using GeWaPro in the following paragraphs:
1. Loading and visualizing the (training) data
2. Setting up MLFlow experiments
3. Retrieving & interpreting test results
4. Using saved models to do predictions
5. Exporting MLFlow models for use

In [ ]:
import time
import timeit
import pandas as pd
import numpy as np
from plotly import graph_objects as go
import plotly.express as px
from datetime import datetime
from IPython.display import HTML
import cufflinks
import numba as nb
from scipy.optimize import least_squares, curve_fit
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.neural_network import MLPRegressor
import mlflow
import os
import tensorflow as tf
import mlflow.keras
import mlflow.sklearn
from gewapro.cache import cache
from gewapro.preprocessing import get_waveforms, train_test_split_cond, smoothen_waveforms, get_and_smoothen_waveforms, select_from_source
from gewapro.functions import (quadratic_arr,
                               fit_parabolas,
                               df_with_fits,
                               _fit_final_slope,
                               combine_and, combine_or,
                               calc_ab)
from gewapro.plotting.base import _fwhm_energy_df
from gewapro.util import name_to_vals, pandas_string_rep, add_notes, combine_cols_with_errors
from gewapro.plotting import (histogram,
                              corr_fig,
                              mlp_reg_fig,
                              plot_transform,
                              energy_histogram,
                              box_plot,
                              plot_predictions,
                              energy_line_plot,
                              add_energy_histogram,
                              combine_line_plots,
                              combined_channel_line_plot,
                              change_combined_line_fig)
from gewapro import plotting
from gewapro.models import regressor_model, train_model, get_model_version_map, ModelInfo, fitted_PCA
import gewapro.models
from gewapro.experiment_flow import run_experiment
import mlflow.pyfunc
import xgboost as xgb
import itertools

cufflinks.go_offline()

# If a new model is added to the saved models, rerun this function below:
gewapro.models.update_validity()

## 1. Loading and visualizing the (training) data
Data for training and testing can be visualized below. See code comments for details.

In [ ]:
# Load in the data
data_g1274_name = "20231110-Na22-d0-12-tz6-ML200-g1274.dat"  # Gate 1274, detector 0 data
data_g1274 = pd.read_csv("data/"+data_g1274_name)
data_g511_name = "20231110-Na22-d0-12-tz6-ML200-g511.dat"  # Gate 511, detector 0 data
data_g511 = pd.read_csv("data/"+data_g511_name)

# Rename Tref to Tfit in columns, not needed if Tfit is already present in columns
data_g511.columns = [("Tfit" if col == "Tref" else col) for col in data_g511.columns]
data_g1274.columns = [("Tfit" if col == "Tref" else col) for col in data_g1274.columns]

# Define a dictionary that holds al data, useful later
data_dict = {
    data_g1274_name : data_g1274,
    data_g511_name  : data_g511,
  # your data name  : your custom data set,

  # For example, a set with start value between (-0.15,0.15) & final value between (0.85,1.15):
    "20231110-Na22-d0-tz6-ML200-g511-tol0.15" : data_g511[(data_g511["s0"]  > -0.15) & \
                                                          (data_g511["s0"]  <  0.15) & \
                                                          (data_g511["s199"] > 0.85) & \
                                                          (data_g511["s199"] < 1.15) ],
}

# Two simple functions that convert the waveform index (x) to time units in ns (t) or back
x_to_t = lambda x: 160-(x*4)
t_to_x = lambda t: (160-t)/4

# Show original data in table form
print(f"Original data ({len(data_g511)} waveforms):")
display(data_g511)
print(f"Dataset length of tolerance .15 data:", len(data_dict["20231110-Na22-d0-tz6-ML200-g511-tol0.15"]), "waveforms")

In [ ]:
# The data is transposed so it can be plotted & used as training data with the get_waveforms(...) function:
df_waveforms = get_waveforms(select_channels=0, source_data=data_g511)
print("Data after waveform transformation (each column is a single waveform for training):")
display(df_waveforms)
print("total waveform count:", len(df_waveforms.columns))

# PLot the data, first 100 waveforms between .85 & .95, and below .85 final data point
display(HTML("<h2>1.1 Displaying the first 100 waveforms (with various imposed conditions):</h2>"))
df_waveforms.iloc[:,-100:].set_index(df_waveforms.index*4).iplot(title="Last 100 waveforms (no conditions)",theme="white",xaxis_title="time [ns]")
dfplot85_95: pd.DataFrame = df_waveforms.loc[:, (df_waveforms.loc[199] > 0.85) & (df_waveforms.loc[199] < 0.95)]
dfplot85_95.iloc[:,:100].set_index(dfplot85_95.index*4).iplot(title="First 100 waveforms with last value between 0.85 and 0.95",theme="white",xaxis_title="time [ns]")
dfplot85: pd.DataFrame = df_waveforms.loc[:,df_waveforms.loc[199] < 0.85]
dfplot85.iloc[:,:100].set_index(dfplot85.index*4).iplot(title="First 100 waveforms with last value below 0.85",theme="white",xaxis_title="time [ns]")

# Plots can be saved to pdf, png or other with the Plotly Figure.write_image(...) method:
fig: go.Figure = dfplot85_95.iloc[:,:100].set_index(dfplot85.index*4).iplot(asFigure=True,theme="white").update_layout(height=400,width=700,margin=dict(l=20, r=20, t=20, b=20),showlegend=False,xaxis_title="time [ns]",yaxis_title="Normalised signal")
fig.write_image("Example_waveforms_plot_85-95perc-finalpoint.pdf")
# fig.show()

In [ ]:
# The distribution of all Tfit timings can be plotted via the histogram(...) function
# By default, a Gaussian fit is included, but it can also be excluded by changing the parameter add_fits from ['Gaussian'] to [].
display(HTML("<h2>1.2 The distribution of all Tfit (initial data) timings:</h2>"))
plot_title = f"Histogram of Tfit timings for dataset '{data_g511_name}'"
histogram(data=data_g511["Tfit"], bins=[-60,30,0.5], mode="Bar", title=plot_title+" (binwidth 0.5, mode 'Bar')", xaxis_title="time [ns]").show()
histogram(data=data_g511["Tfit"], bins=[-60,30,2], mode="Line", title=plot_title+" (binwidth 2.0, mode 'Line')", xaxis_title="time [ns]").show()
hist = histogram(data=data_g511["Tfit"], bins=[-60,30,1], add_fits=["Inv. Quadratic"], title=plot_title+" (binwidth 1.0, mode 'DEFAULT')", xaxis_title="time [ns]")
hist.show()

In [ ]:
# Changing the default plotting settings can be done using the gewapro.plotting.settings(...) function
print("Unchanged default settings:", plotting.settings.reset())
# From here we set the default_plot_mode for histograms to 'Line':
plotting.settings(default_plot_mode='Line')

## 2. Setting up MLFlow experiments

The ``run_experiment(...)`` function can be used to train a model and run & log it inside an MLFlow experiment.

**!!! First, run ``mlflow ui`` in a terminal in this folder, so that the experiment runs with trained models can be saved !!!**

In [ ]:
# After this, a tracking URI can be set
tracking_uri = "http://127.0.0.1:5000"  # It is known that on Windows the default /localhost/ URI is slow, so use 127.0.0.1 instead
mlflow.set_tracking_uri(tracking_uri)
print("[INFO] Set MLFlow tracking URI to:",tracking_uri)

### 2.1 Single experiment

In [ ]:
# Run single experiment
model_type = "SKLearn"  
uniform_test_set = [5000,6000,7000,8000,9000,10000,11000,12000,13000,15000]
data_g511_tol015 = "20231110-Na22-d0-tz6-ML200-g511-tol0.15"

result_single_exp = run_experiment(
    experiment_name = "Test Experiment",
  # run_name = custom run name,           # The run name defaults to a combination of training data, date & time
    data = data_dict[data_g511_tol015],   # Using the data dict here to get the DataFrame "data_g511", not yet turned into waveforms
    data_name = data_g511_tol015,         # Name of the data, used for tracking/logging
    select_channels = [] ,                # All channels available, so only channel 0 for this data set
    select_energies = (5000,15000),       # From arbitrary units 5000 - 15000
    include_energy = False,               # Whether to include final amplitude in training, default: False
    pca_components = None,                # No PCA components, this means no PCA is used
    pca_method = TruncatedSVD,            # Which PCA method to use (if components is set), by default sklearn.decomposition.PCA
    model_type = model_type,              # "SKLearn" or "TensorFlow" or "XGBoost"
    test_size = 0.3,                      # Testing set size compared to total data set
    uniform_test_set = uniform_test_set,  # Uniform extra test sets of energies 5000-6000, 6000-7000, etc.
    registered_model_name = "TestModel",  # Name of the model in the MLFlow registry, set this to "auto" to not worry about it
  # Only valid for Neural Networks:
    hidden_layers = [23],                 # Hidden layers and amount of neurons, e.g. [16,34] is a two-layer model with 16 and 34 neurons each
    alpha = 1e-4,                         # Alpha regularisation parameter or 'training strictness': the lower, the stricter
    max_iterations = 1000,                # Max number of training iterations, by default: 2000
    activation = "relu",                  # Neuron activation function: "relu" is the rectified linear function: 0 for negative x, x for positive x
)

# The run_experiment function outputs a prediction histogram by default, which can be shown:
result_single_exp.show()
print("Experiment result parameters:", result_single_exp._params)

### 2.2 Loops of experiments
A bunch of experiment repetitions can also be done with for-loops. For example, in the case of XGBoosted Regression Trees, testing the amount of regressors, regularisation, max tree depth, branching factor etc.

In [ ]:
# Run group of experiments
model_type = "XGBoost"
uniform_test_set = [5000,6000,7000,8000,9000,10000] # Test sets with energy ranges 5000-6000, 6000-7000, 7000-8000, 8000-9000 & 9000-10000, of equal sample size
data_g511_tol015 = "20231110-Na22-d0-tz6-ML200-g511-tol0.15"

# The lists
iterations = 2     # 2 iterations per model configuration
pca_comps_list = [16, 32, None]
max_tree_depth_list = [20, 30]
number_of_regressors_list = [3, 5]
total_iterations = iterations*len(pca_comps_list)*len(max_tree_depth_list)*len(number_of_regressors_list)
print(f"Received {total_iterations} (= {iterations}*{len(pca_comps_list)}*{len(max_tree_depth_list)}*{len(number_of_regressors_list)}) experiments to run.")

# The group of experiment runs, takes about 1 min per iteration -> 24 iterations ~ 24 minutes
i = 0
for pca_comps in pca_comps_list:
    for tree_depth in max_tree_depth_list:
        for regressor_count in number_of_regressors_list:
            for iteration in range(iterations):
                i += 1
                print(f"Starting experiment {i}/{total_iterations} (iteration {iteration+1}/{iterations})...")
                result_single_exp = run_experiment(
                    experiment_name = "Looped Test Experiment",
                  # run_name = custom run name,             # The run name defaults to a combination of training data, date & time
                    data = data_dict[data_g511_tol015],     # Using the data dict here to get the DataFrame "data_g511", not yet turned into waveforms
                    data_name = data_g511_tol015,           # Name of the data, used for tracking/logging
                    select_channels = [] ,                  # All channels available, so only channel 0 for this data set
                    select_energies = (5000,15000),         # From arbitrary units 5000 - 15000
                    include_energy = False,                 # Whether to include final amplitude in training, default: False
                    pca_components = pca_comps,             # No PCA components, this means no PCA is used
                    pca_method = PCA,                       # Which PCA method to use (if components is set), by default sklearn.decomposition.PCA
                    model_type = model_type,                # "SKLearn" or "TensorFlow" or "XGBoost"
                    test_size = 0.3,                        # Testing set size compared to total data set
                    uniform_test_set = uniform_test_set,    # Uniform extra test sets of energies 5000-6000, 6000-7000, etc.
                    registered_model_name = "XGBTestModel", # Name of the model in the MLFlow registry, set this to "auto" to not worry about it
                  # Only valid for Regression Trees:
                    max_depth = tree_depth,
                    n_estimators = regressor_count,
                    max_leaves = 0,
                )
print("Final experiment distribution graph:")
result_single_exp.show()

# New models were added to the saved models, so we rerun the update_validity function
gewapro.models.update_validity()

## 3. Retrieving & interpreting test results
Now, after these first 2 experiments, going to [127.0.0.1:5000](http://127.0.0.1:5000) in browser, there should be two experiments shown in the left bar: *Test Experiment* & *Looped Test Experiment*. Clicking on these, all individual runs can be seen, with the models saved there also available. The *Test Experiment* should have only one trained *TestModel* (or as many times as you ran the single experiment), while the *Looped Test Experiment* should have 24 models available, called *XGBTestModel*.

The performance of these models can be compared in a graph. All one needs is the experiment number, which is in the URL after ``experiments/``, such as in the URL
``http://127.0.0.1:5000/#/experiments/595346769839476301?searchFilter=&orderByKey=...``. In this case, the experiment number is ``595346769839476301``.

Graphs that are visually intuitive to use here are box plots, or distribution plots. The ``box_plot(...)`` function can retrieve the experiment results and plot the performance of all models in a single plot. The performance metric can be provided yourself via the ``y`` argument.

In [ ]:
# Put here the experiment ids from the URL:
single_exp_id = 595346769839476301 #...
looped_exp_id = 755987746823550296 #...

# Show performance box plot of the two experiments
box_plot(exp_id=[single_exp_id, looped_exp_id], x="PCA components", y="FWHM Test", color="Model name", hover_name="model_version").show()

# Show performance box plot of the XGBTestModel looped experiment
box_plot(exp_id=[looped_exp_id],
         x="PCA components",
         y="FWHM Test",
         color="Max tree depth",
         facet_row="Number of estimators",
         height=700,
         ignore_vals={"Number of estimators":None},
         hover_name="model_version").show()

box_plot(exp_id=[looped_exp_id],
         x="PCA components",
         y="Overtraining factor",
         color="Max tree depth",
         facet_row="Number of estimators",
         height=700,
         ignore_vals={"Number of estimators":None},
         hover_name="model_version",
         load_cols="FWHM Test").show()

In [ ]:
# Advanced metrics can also be made and used in the box plots
def square_FWHM_metric(df: pd.DataFrame) -> pd.Series:
    ranges = {"5000-6000":13.3579,"6000-7000":11.841,"7000-8000":10.3735,"8000-9000":10.2916,"9000-10000":9.4387,"10000-11000":11.2823,"11000-12000":7.0638}
    return sum([(df[f"metrics.Uniform test FWHM E{e_range}"]/e_val)**2 for e_range,e_val in ranges.items()])

def linear_FWHM_metric(df: pd.DataFrame) -> pd.Series:
    ranges = {"5000-6000":13.3579,"6000-7000":11.841,"7000-8000":10.3735,"8000-9000":10.2916,"9000-10000":9.4387,"10000-11000":11.2823,"11000-12000":7.0638}
    return sum([df[f"metrics.Uniform test FWHM E{e_range}"]/e_val for e_range,e_val in ranges.items()])
linear_FWHM_metric.FWHM = square_FWHM_metric.FWHM = 8

box_plot([single_exp_id],
         x="PCA components",
         y=square_FWHM_metric,
         color="Hidden layers").show()

box_plot([single_exp_id],
         x="PCA components",
         y=linear_FWHM_metric,
         color="Hidden layers").show()

## 4. Using saved models to do predictions
Saved models from experiments can be retrieved with the ``ModelInfo`` class, which can be instantiated with the ``from_database`` method.

### 4.1 Retrieving saved models

In [ ]:
# First run 'mlflow ui' in a terminal, otherwise this will not work!!!
test_model_1_info = ModelInfo.from_database(model_name="TestModel",model_version=1)
test_model_12_info = ModelInfo.from_database(model_name="XGBTestModel",model_version=12)
print(test_model_1_info)
print(test_model_12_info)

# The models themselves and their PCA classes can be extracted
model_12 = test_model_12_info.model
pca_model_12 = test_model_12_info.get_transformer()
print(model_12)
pca_model_12

In [ ]:
# Predictions can be done with plot_predictions(...), note that the PCA parameters must be exactly the same as before, so fitted on the same training data
model_version = 22  # Can be varied to plot for different model versions
pca_fit_model = fitted_PCA(model_name = "XGBTestModel",
                           model_version = model_version,
                           waveforms = get_waveforms(source_data=data_dict["20231110-Na22-d0-tz6-ML200-g511-tol0.15"]))

# Test predictions on other data set (1274)
plot_predictions(on_data = data_g1274_name,
                 energy_range = (),
                 model_version = model_version,
                 data_dict = data_dict,
                 model_name = "XGBTestModel",
                 PCA_fit = pca_fit_model,
                 plot = "Histogram")

# Same model but on original data set (511), should be better than new data set
plot_predictions(on_data = data_g511_name,
                 energy_range = (),
                 model_version = model_version,
                 data_dict = data_dict,
                 model_name = "XGBTestModel",
                 PCA_fit = pca_fit_model,
                 plot = "Histogram")

# SKLearn NN model on other data set (1274)
plot_predictions(on_data = data_g1274_name,
                 energy_range = (),
                 model_version = 1,
                 data_dict = data_dict,
                 model_name = "TestModel",
                 plot = "Histogram")

### 4.2 Energy line plots & energy histograms

In [ ]:
# Energy corrections and histogram
e_corrections = {0: calc_ab(4477,11197),
                 1: calc_ab(4623,11538),
                 2: calc_ab(4212,10512),
                 3: calc_ab(4672,11662),
                 4: (1,0),  # LaBr channel
                 5: calc_ab(1582,3948),
                 6: (1,0),  # LaBr channel
                 7: calc_ab(4747,11866),
                 8: calc_ab(4303,10727),
                 9: calc_ab(4750,11861),
                 10:calc_ab(4113,10268),
                 11: calc_ab(4474,11157)}

fig_ehist = energy_histogram(data_g511_name, data_dict, select_energies=(0,1300), bins=[0,1400,2], correct_energy=e_corrections[0], xaxis_title="Energy [keV]",
                             title=f"Predictions of XGBTestModel on channel 0 (trained on channel 0)", colors=["rgba(0,0,0,0.5)"])

# Performance of a model on the energy range of the data can be visualized with energy_line_plot
fig_eline = energy_line_plot(on_data = data_g1274_name,
                             start = 50,
                             end = 1250,
                             step = 100,
                             model_version = model_version,
                             data_dict = data_dict,
                             model_name = "XGBTestModel",
                             PCA_fit = pca_fit_model,
                             hist_limit = 34,
                             correct_energy = e_corrections[0])

add_energy_histogram(fig_eline=fig_eline, fig_ehist=fig_ehist)

### 4.3 More plotting tools
Combined channel line plots can combine predictions of different channels.

## 5. Exporting MLFlow models for use
Soon to be added...

In [ ]:
...

## A. Appendix: Caching

In [ ]:
# Caching is used a lot in the functions under the hood
# Info on cache can be gotten with the .cache_info() method for all functions that have a @cache decorator
print(energy_line_plot.cache_info())

---